In [4]:
import pandas as pd

In [5]:
#reading the csv file using pandas read_csv function
data = pd.read_csv('gapminder.csv', thousands=',', index_col='Year')
#displaying the content of the dataset
data.head()

,Country,life,population,income,region
Year,,,,,
1800,Afghanistan,28.211000,3280000.0,603.0,South Asia
1801,Afghanistan,28.200753,NaN,603.0,South Asia
1802,Afghanistan,28.190507,NaN,603.0,South Asia
1803,Afghanistan,28.180260,NaN,603.0,South Asia
1804,Afghanistan,28.170013,NaN,603.0,South Asia


In [6]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [7]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.models import NumeralTickFormatter
p = figure(
    height=300,x_axis_type='log',                       #using 'log' to make the graph make more sense
    x_range=(100,100000), y_range=(0,100)
)
p.circle(x=data.loc[2010].income, y=data.loc[2010].life)
p.xaxis[0].formatter=NumeralTickFormatter(format='$0,')  #using NumeralTickFormatter to change the format
p.xaxis.axis_label = "Income"
p.xaxis.axis_label_text_color = "#00008B"
p.xaxis.axis_label_text_font_style = "bold"
p.yaxis.axis_label = "Life Expectancy"
p.yaxis.axis_label_text_color = "#00008B"
p.yaxis.axis_label_text_font_style = "bold"
show(p)

In [8]:
from bokeh.models import ColumnDataSource #allows to pass df or dictionary

In [9]:
source = ColumnDataSource(dict(
        x=data.loc[2010].income, 
        y=data.loc[2010].life,
    country=data.loc[2010].Country 
)) #including country in the dictionary for further analysis
source.column_names#checking the names of the columns in the source

['x', 'y', 'country']

In [10]:
#creating the PLOT_OPS that could be reused later
PLOT_OPS = dict(
    height=400, x_axis_type='log',
    x_range=(100,100000), y_range=(0,100),
)

In [11]:
from bokeh.models import HoverTool
from bokeh.models import NumeralTickFormatter
hover = HoverTool(tooltips='@country',show_arrow=False)
p = figure(tools=[hover], **PLOT_OPS)
p.circle(x='x',y='y', size=10, alpha=0.6, source=source) #transform is used to use the function to update the fields
p.xaxis[0].formatter=NumeralTickFormatter(format='$0,')
p.xaxis.axis_label = "Income"
p.xaxis.axis_label_text_color = "#00008B"
p.xaxis.axis_label_text_font_style = "bold"
p.yaxis.axis_label = "Life Expectancy"
p.yaxis.axis_label_text_color = "#00008B"
p.yaxis.axis_label_text_font_style = "bold"
show(p)

In [12]:
source = ColumnDataSource(dict(
        x=data.loc[2010].income, 
        y=data.loc[2010].life,
        country=data.loc[2010].Country,
        population=data.loc[2010].population,
        region=data.loc[2010].region
))   #including region in the dictionary for further analysis
source.column_names #checking the names of the columns in the source

['x', 'y', 'country', 'population', 'region']

In [13]:
from bokeh.io import push_notebook #allows us to write back to the notebook

#creating a function which will update the data for every year from 1950 to 2014
#starting with year 2010
def update(year):
    new_data=dict(                          #creating a new data dictionary which will be used to update the data year wise
    x=data.loc[year].income,                #storing the values for the income data from the next year
    y=data.loc[year].life,                  #storing the values for the life expectancy data from the next year
    country=data.loc[year].Country,         #storing the values for the country data from the next year
    region=data.loc[year].region,           #storing the values for the region data from the next year
    population=data.loc[year].population    #storing the values for the population from the next year
    )
    source.data=new_data                    #storing the dictionary in a n from the next year
    p.title.text=str(year)                  #giving the title to the plot for the year
    push_notebook()                         #Updating plots in the notebook output cells with new data or property values

from bokeh.models import LinearInterpolator #LinearInterpolator is used to set the range of the plots without having to manually write it
    
size_mapper= LinearInterpolator(                
    x=[data.population.min(),data.population.max()],   #the min and the max function are used to set the range of the x-axis
    y=[5,50]
    )

from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Spectral6

color_mapper=CategoricalColorMapper(                #This function is used to give colors to the plots based on certain attributes
    factors= list(data.region.unique()),
    palette=Spectral6,                             #using the pre-defined color code
)
p = figure(
    title=str(2010), toolbar_location='above',            #location above is used for defining where the year title will be
    **PLOT_OPS)
p.circle(x='x',y='y', 
         size={'field':'population', 'transform': size_mapper},  #field tells which field is to be updated
         color={'field': 'region', 'transform': color_mapper},   #transform is used to use the function to update the fields
         alpha=0.6, 
         source=source,
        legend='region')
p.xaxis.axis_label = "Income"
p.xaxis.axis_label_text_color = "#00008B"
p.xaxis.axis_label_text_font_style = "bold"
p.yaxis.axis_label = "Life Expectancy"
p.yaxis.axis_label_text_color = "#00008B"
p.yaxis.axis_label_text_font_style = "bold"
p.legend.border_line_color=None
p.right.append(p.legend[0])
p.xaxis[0].formatter=NumeralTickFormatter(format='$0,')
show(p, notebook_handle=True)

In [14]:
from ipywidgets import interact
interact(update, year=(1950, 2014,1))

interactive(children=(IntSlider(value=1982, description='year', max=2014, min=1950), Output()), _dom_classes=(…

<function __main__.update(year)>